In [11]:
import csv
import openpyxl
import json
import random
import os
import numpy as np
import math
import datetime
import PIL
from PIL import ImageOps
from PIL import ImageChops
from PIL import Image
from fpdf import FPDF
from IPython.display import IFrame

In [12]:
xlsxFile = "All Hat And No Cattle.xlsx"
imgUrl = "https://raw.githubusercontent.com/nicfreeman1209/botc_other/main/all_hat_and_no_cattle/images/"
logoUrl = imgUrl + "logo/" + "cowboy_hat.png"
author = "Nic"
today = datetime.datetime.now().strftime("%Y-%m-%d")

In [13]:
# load characters from csv, compile json
teamCol = 0
nameCol = 1
abilCol = 2
reminderCol = 3
globalRemCol = 4
setupCol = 5
night1Col = 6
night1RemCol = 7
night2Col = 8
night2RemCol = 9

night1Order = []
night2Order = []
characters = []

workbook = openpyxl.load_workbook(xlsxFile)
sheet = workbook.active
reader = sheet.iter_rows()

topRow = next(reader)
title = topRow[1].value #+ " v1"
next(reader)

for _row in reader:
    row = ["" if cell.value is None else str(cell.value) for cell in _row]
    if row[teamCol] != "":
        currentTeam = row[teamCol]
    
    if row[nameCol] != "" and row[nameCol] != "Apatheist":
        _id = row[nameCol].lower().replace(" ","_")
        reminders = [x.strip() for x in row[reminderCol].split(",")]
        if reminders[0]=="":
            reminders = []
        globalReminders = [x.strip() for x in row[globalRemCol].split(",")]
        if globalReminders[0]=="":
            globalReminders = []
        characters.append({"id": _id,
                           "image": imgUrl+"compiled/"+_id+".png",
                           "edition": "custom",
                           "firstNight": 0,
                           "firstNightReminder": "",
                           "otherNight": 0,
                           "otherNightReminder": "",
                           "reminders": reminders,
                           "remindersGlobal": globalReminders,
                           "setup": int(row[setupCol]),
                           "name": row[nameCol],
                           "team": currentTeam,
                           "ability": row[abilCol].strip()})
    if row[night1Col] != "":
        night1Order.append((row[night1Col], row[night1RemCol]))
    if row[night2Col] != "":
        night2Order.append((row[night2Col], row[night2RemCol]))

night1Order.reverse()  
night2Order.reverse()  
nights = [(night1Order, "firstNight"), (night2Order, "otherNight")]
for order,prefix in nights:
    n = 10 # buffer for demon/minion info etc
    for name,reminder in reversed(order):
        for character in characters:
            if name==character["name"]:
                character[prefix] = n
                character[prefix+"Reminder"] = reminder
                n += 1

script = []
script.append({"id":"_meta", "name":title, "logo":logoUrl, "author":author, "date":today})
for character in characters:
    script.append(character)

with open(title+'.json', 'w') as f:
    json.dump(script, f, indent=4)
    
script

[{'id': '_meta',
  'name': 'All Hat And No Cattle',
  'logo': 'https://raw.githubusercontent.com/nicfreeman1209/botc_other/main/all_hat_and_no_cattle/images/logo/cowboy_hat.png',
  'author': 'Nic',
  'date': '2022-12-19'},
 {'id': 'sweep',
  'image': 'https://raw.githubusercontent.com/nicfreeman1209/botc_other/main/all_hat_and_no_cattle/images/compiled/sweep.png',
  'edition': 'custom',
  'firstNight': 13,
  'firstNightReminder': 'Ability - first night.',
  'otherNight': 0,
  'otherNightReminder': '',
  'reminders': [],
  'remindersGlobal': [],
  'setup': 0,
  'name': 'Sweep',
  'team': 'townsfolk',
  'ability': 'You start knowing the length of the longest row of neighbouring townsfolk.'},
 {'id': 'understudy',
  'image': 'https://raw.githubusercontent.com/nicfreeman1209/botc_other/main/all_hat_and_no_cattle/images/compiled/understudy.png',
  'edition': 'custom',
  'firstNight': 16,
  'firstNightReminder': 'Ability - each night.',
  'otherNight': 20,
  'otherNightReminder': 'Ability - 

In [14]:
# compile images from masks & base texs
imageDir = os.path.join(os.path.abspath(''), "images")
blueTex = PIL.Image.open(os.path.join(imageDir, "bases", "blue.png"))
redTex = PIL.Image.open(os.path.join(imageDir, "bases", "red.png"))
blueTex = blueTex.convert("RGBA")
redTex = redTex.convert("RGBA")
random.seed(1)
s = 295*2 # sidelength

def SelectPortionOfImage(im):
    # select a random square from within an image
    w, h = im.width, im.height
    w_offset, h_offset = random.randint(0,w-s), random.randint(0,h-s)
    im2 = im.copy()
    if random.random()>0.5:
        im2.transpose(PIL.Image.FLIP_LEFT_RIGHT)
    if random.random()>0.5:
        im2.transpose(PIL.Image.FLIP_TOP_BOTTOM)
    im2 = im2.crop((w_offset, h_offset, w_offset+s, h_offset+s))
    return im2

def ChopImageInHalf(im):
    w, h = im.width, im.height
    im = im.crop((0,0,w/2,h))
    return im

def ConcatImagesW(im1, im2):
    dst = Image.new('RGBA', (im1.width + im2.width, im1.height))
    dst.paste(im1, (0, 0))
    dst.paste(im2, (im1.width, 0))
    return dst

def FetchFilledPartOfImage(im):
    # crop the non-blank part (of a mask)
    im = im.convert("RGBA")
    data = np.asarray(im)
    alphas = data[:,:,3]
    nzRows = np.where(data.any(axis=0))[0]
    nzCols = np.where(data.any(axis=1))[0]
    l,r = nzRows[0], nzRows[-1]
    b,t = nzCols[0], nzCols[-1]
    o_x = (l+r)/2
    o_y = (t+b)/2
    o = max(r-l, t-b)/2
    o *= math.sqrt(2) * 1.08 # padding
    im = im.crop((o_x-o, o_y-o, o_x+o, o_y+o))
    im = im.resize((s,s), Image.ANTIALIAS)
    return im
    
for character in characters:
    name = character['id']
    imgFilename = name + ".png"
    imgPath = os.path.join(imageDir, "masks", imgFilename)
    team = character["team"] 
    
    if team in ["townsfolk", "outsider"]:
        baseTex = SelectPortionOfImage(blueTex)         
    elif team in ["minion", "demon"]:
        baseTex = SelectPortionOfImage(redTex)
    elif team == "traveler":
        _blueTex = ChopImageInHalf(SelectPortionOfImage(blueTex))
        _redTex = ChopImageInHalf(SelectPortionOfImage(redTex))
        baseTex = ConcatImagesW(_blueTex, _redTex)

    rawMask = PIL.Image.open(imgPath) 
    mask = FetchFilledPartOfImage(rawMask)
    PIL.ImageChops.offset(mask, 0, int(-0.045*s)) # compensate for css/bra1ntool weirdness
    
    blank = PIL.Image.new("RGBA", (mask.width, mask.height), (0,0,0,0))
    icon = PIL.Image.composite(baseTex, blank, mask)
    icon = icon.resize((300,300), Image.ANTIALIAS)
    icon.save(os.path.join(imageDir, "compiled", name.lower()+".png"))

In [27]:
# create pdf
pdf = FPDF(format="A4", unit="mm", orientation="P") # 210 x 297
pdf.set_margin(0)

fontSize = 10
fontName = "Helvetica"

pdf.add_page()
pdf.set_xy(0,10)
pdf.set_font(fontName, 'B', 16)
pdf.cell(w=210, h=0, align='C', txt=title, border=0)

pt = 0.36
pdf.set_font(fontName, '', fontSize)
y = 20
lMargin = 12
colWidth = 95
midMargin = 2
imSize = 12
x1 = lMargin
x2 = lMargin + colWidth + midMargin
x = x2

teamColors = {
    "townsfolk": (20,118,212),
    "outsider": (20,118,212),
    "minion": (149,18,36),
    "demon": (149,18,36),
    "traveler": (100,100,100),
}

def AddCharacter(x, y, character):
    if character:
        imageFile = os.path.join(imageDir, "compiled", character["id"].lower()+".png")
        pdf.image(imageFile, x=x, y=y, w=imSize, h=imSize)
        pdf.set_xy(x+imSize,y)
        pdf.set_font(fontName, '', fontSize)
        pdf.multi_cell(w=colWidth-imSize, h=3.5, align='L', txt="**" + character["name"] + ":** " + character["ability"], border=0, markdown=True)
    y += fontSize*pt*5.5
    return y
    
def AddTeamRect(y1,y2, teamName="TEAM"):
    w = lMargin/3
    h = y2-y1-fontSize*pt  
    pdf.rect(x=lMargin/3, y=y1-fontSize*pt/2, w=w, h=h)
    pdf.set_xy(lMargin/3, h+y1-fontSize*pt/2)
    with FPDF.rotation(pdf, angle=90, x=pdf.get_x(), y=pdf.get_y()):
        pdf.set_xy(pdf.get_x(),pdf.get_y()+lMargin/3/2+0.2)
        color = teamColors[teamName]
        pdf.set_text_color(*color)
        pdf.set_font(fontName, '', fontSize-1)
        pdf.cell(w=h,h=0,align="C",txt=teamName.upper())
        pdf.set_text_color(0,0,0)
        pdf.set_font(fontName, '', fontSize)
    
prevTeam = "townsfolk"
y1 = y
y2 = None

for character in characters:
    if character["team"] != prevTeam:
        if x == x1:
            y = AddCharacter(x,y,None)
        x = x2
        y += fontSize*pt*0.75
        AddTeamRect(y1,y, prevTeam)
        prevTeam = character["team"]
        if character["team"] == "traveler":
            pdf.add_page()
            pdf.set_xy(0,10)
            y = 15
        y1 = y
    if x == x2:
        x = x1
        AddCharacter(x, y, character)
    else:
        x = x2
        y = AddCharacter(x, y, character)

if x == x1:
    y = AddCharacter(x,y,None)
AddTeamRect(y1,y, "traveler")
y += fontSize*pt*3

x = x1
pdf.set_xy(x1+imSize, y)
pdf.cell(w=colWidth, h=0, align='L', txt="**First Night**", border=0, markdown=True)
y += fontSize*pt

for name,reminder in reversed(night1Order):
    y += fontSize*pt
    pdf.set_xy(x+imSize,y)
    pdf.cell(w=colWidth, h=0, align="L", txt="**"+name+"**: "+reminder, markdown=True)
y += fontSize*pt*3
pdf.set_xy(x1+imSize, y)
pdf.cell(w=colWidth, h=0, align='L', txt="**Other Night**", border=0, markdown=True)
y += fontSize*pt

for name,reminder in reversed(night2Order):
    y += fontSize*pt
    pdf.set_xy(x1+imSize,y)
    pdf.cell(w=colWidth, h=0, align="L", txt="**"+name+"**: "+reminder, markdown=True)

pdf.set_xy(x1+imSize, 297-15)
pdf.set_text_color(150,150,150)
pdf.cell(w=colWidth, h=0, align="L", txt=today)
    
filename = title + '.pdf'
pdf.output(filename,'F')

IFrame(filename, width=1000, height=600)